In [1]:
from scipy import misc
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import glob
from PIL import Image
import os
ellipse_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')
line_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')
Diamond_fs = glob.glob('/home/abdullah/PycharmProjects/Pattern/Ellipse_TS/*.bmp')

x = np.array([np.array(Image.open(fname)) for fname in ellipse_fs])
x = np.concatenate((x,np.array([np.array(Image.open(fname)) 
                                for fname in line_fs])),axis=0)
x = np.concatenate((x,[np.array(Image.open(fname)) 
                    for fname in Diamond_fs]),axis=0).astype('float32')

y=np.zeros((255,)).astype('int')
y[0:85]=0
y[85:170]=1
y[170:]=2
b = np.zeros((255, 3))
b[np.arange(255), y] = 1
y=b
X_train, X_test, y_train, y_test = train_test_split\
    (x, y, test_size=0.1)
X_train, X_eval, y_train, y_eval = train_test_split\
    (X_train, y_train, test_size=0.12)

In [2]:
def train():
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    history = []
    iterep = 500
    for i in range(iterep * 30):
        idx = np.random.choice(np.arange(len(y_train)), 100, replace=False)
        x_batch = X_train[idx]
        y_batch = y_train[idx]
        
        sess.run(train_step,
                 feed_dict={'x:0': x_batch, 
                            'y:0': y_batch, 
                            'phase:0': 1})
        if (i + 1) %  iterep == 0:
            epoch = (i + 1)/iterep
            tr = sess.run([loss, accuracy], 
                          feed_dict={'x:0': X_train,
                                     'y:0': y_train,
                                     'phase:0': 1})
            t = sess.run([loss, accuracy], 
                         feed_dict={'x:0': X_eval,
                                    'y:0': y_eval,
                                    'phase:0': 0})
            history += [[epoch] + tr + t]
            print history[-1]
    return history

In [3]:
def dense(x, size, scope):
    return tf.contrib.layers.fully_connected(x, size, 
                                             activation_fn=None,
                                             scope=scope)

def dense_relu_drop(x, phase,size, scope):
    with tf.variable_scope(scope):
        h1 = tf.contrib.layers.fully_connected(x, size, 
                                               activation_fn=tf.nn.relu,
                                               scope='dense')
        dropout_2 =tf.layers.dropout(inputs=h1, rate=0.4,
                                     training=phase)

        
        return dropout_2

In [4]:
x = tf.placeholder('float32', (None, 256,256), name='x')
y = tf.placeholder('float32', (None,3), name='y')
phase = tf.placeholder(tf.bool, name='phase')
input_layer = tf.reshape(x, [-1, 256* 256])
# dense = tf.layers.dense(inputs=input_layer, units=32,activation=tf.nn.relu)
# bn1= tf.layers.batch_normalization(dense, 
#                                       center=True, trainable=True,scale=True)
# dropout_1 =tf.layers.dropout(inputs=bn1, rate=0.4,training=mode == tf.estimator.ModeKeys.TRAIN)
# dense2 = tf.layers.dense(inputs=dropout_1, units=64,activation=tf.nn.relu)
# dropout_2 =tf.layers.dropout(inputs=dense2, rate=0.4,training=mode == tf.estimator.ModeKeys.TRAIN)
# logits = tf.layers.dense(inputs=dropout_2, units=3)
h1 = dense_relu_drop(input_layer, phase,32,'layer1')
h2 = dense_relu_drop(h1, phase,64, 'layer2')
h3 = dense_relu_drop(h1, phase,64, 'layer3')
logits = dense(h3, 3, 'logits')

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(y, 1), tf.argmax(logits, 1)), 
            'float32'))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [5]:
import sys
sys.path.append('/home/abdullah/PycharmProjects/Pattern') # point to your tensorflow dir
from utils import show_graph
show_graph(tf.get_default_graph().as_graph_def())

<iframe seamless style="width:1200px;height:620px;border:0" srcdoc="
        <script>
          function load() {
            document.getElementById(&quot;graph0.408843781615&quot;).pbtxt = 'node {\n  name: &quot;x&quot;\n  op: &quot;Placeholder&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;shape&quot;\n    value {\n      shape {\n        dim {\n          size: -1\n        }\n        dim {\n          size: 256\n        }\n        dim {\n          size: 256\n        }\n      }\n    }\n  }\n}\nnode {\n  name: &quot;y&quot;\n  op: &quot;Placeholder&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;shape&quot;\n    value {\n      shape {\n        dim {\n          size: -1\n        }\n        dim {\n          size: 3\n        }\n      }\n    }\n  }\n}\nnode {\n  name: &quot;phase&quot;\n  op: &quot;Placeholder&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_BOOL\n    }\n  }\n  attr {\n    key: &quot;shape&quot;\n    value {\n      shape {\n        unknown_rank: true\n      }\n    }\n  }\n}\nnode {\n  name: &quot;Reshape/shape&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_INT32\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_INT32\n        tensor_shape {\n          dim {\n            size: 2\n          }\n        }\n        tensor_content: &quot;\\377\\377\\377\\377\\000\\000\\001\\000&quot;\n      }\n    }\n  }\n}\nnode {\n  name: &quot;Reshape&quot;\n  op: &quot;Reshape&quot;\n  input: &quot;x&quot;\n  input: &quot;Reshape/shape&quot;\n  attr {\n    key: &quot;T&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;Tshape&quot;\n    value {\n      type: DT_INT32\n    }\n  }\n}\nnode {\n  name: &quot;layer1/dense/weights/Initializer/random_uniform/shape&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;_class&quot;\n    value {\n      list {\n        s: &quot;loc:@layer1/dense/weights&quot;\n      }\n    }\n  }\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_INT32\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_INT32\n        tensor_shape {\n          dim {\n            size: 2\n          }\n        }\n        tensor_content: &quot;\\000\\000\\001\\000 \\000\\000\\000&quot;\n      }\n    }\n  }\n}\nnode {\n  name: &quot;layer1/dense/weights/Initializer/random_uniform/min&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;_class&quot;\n    value {\n      list {\n        s: &quot;loc:@layer1/dense/weights&quot;\n      }\n    }\n  }\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_FLOAT\n        tensor_shape {\n        }\n        float_val: -0.00956598389894\n      }\n    }\n  }\n}\nnode {\n  name: &quot;layer1/dense/weights/Initializer/random_uniform/max&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;_class&quot;\n    value {\n      list {\n        s: &quot;loc:@layer1/dense/weights&quot;\n      }\n    }\n  }\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_FLOAT\n        tensor_shape {\n        }\n        float_val: 0.00956598389894\n      }\n    }\n  }\n}\nnode {\n  name: &quot;layer1/dense/weights/Initializer/random_uniform/RandomUniform&quot;\n  op: &quot;RandomUniform&quot;\n  input: &quot;layer1/dense/weights/Initializer/random_uniform/shape&quot;\n  attr {\n    key: &quot;T&quot;\n    value {\n      type: DT_INT32\n    }\n  }\n  attr {\n    key: &quot;_class&quot;\n    value {\n      list {\n        s: &quot;loc:@layer1/dense/weights&quot;\n      }\n    }\n  }\n  attr {\n    key: &quot;dtype&quot;\n    value {\n

In [6]:
history = train()

[1, 1.085876, 0.35323384, 1.4539725, 0.2857143]


[2, 1.0816145, 0.35323384, 1.5081863, 0.2857143]


[3, 1.0874186, 0.35323384, 1.7716792, 0.2857143]


[4, 1.0819426, 0.35323384, 1.8819885, 0.2857143]


[5, 1.0821347, 0.35323384, 1.6105683, 0.2857143]


...

[9, 1.0773226, 0.35323384, 2.716428, 0.2857143]


[10, 1.076493, 0.35323384, 2.8055267, 0.2857143]


[11, 1.0872111, 0.35323384, 2.9833264, 0.2857143]


[12, 1.0821431, 0.35323384, 3.0350425, 0.2857143]


[13, 1.0819731, 0.35323384, 3.1183798, 0.2857143]


In [2]:
def dn_model_fn(features, labels, mode):
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 256* 256* 1])
    dense = tf.layers.dense(inputs=input_layer, units=32,activation=tf.nn.relu)
    bn1= tf.layers.batch_normalization(dense, 
                                          center=True, trainable=True,scale=True)
    # dropout_1 =tf.layers.dropout(inputs=bn1, rate=0.4,training=mode == tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(inputs=bn1, units=64,activation=tf.nn.relu)
    # dropout_2 =tf.layers.dropout(inputs=dense2, rate=0.4,training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dense2, units=3)
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(
        indices=tf.cast(labels, tf.int32), depth=3)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)
    
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
print y.dtype
pat_classifier = tf.estimator.Estimator(
      model_fn=dn_model_fn,model_dir="/tmp/pat_dump")
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": X_train},
      y=y_train,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
pat_classifier.train(
      input_fn=train_input_fn,
      steps=200,
      hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": X_eval},
      y=y_eval,
      num_epochs=1,
      shuffle=False)
eval_results = pat_classifier.evaluate(input_fn=eval_input_fn)

print(eval_results)

float64
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/pat_dump', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/pat_dump/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/pat_dump/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0